In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from os import listdir

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [3]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

ER Data Entropy Caluclation

In [4]:
DAMAN = "../../jalend/ParsedData/Daman/"
MANIPUR = "../../jalend/ParsedData/Manipur/"
MEGHALAYA = "../../jalend/ParsedData/Meghalaya/"
NAGALAND = "../../jalend/ParsedData/Nagaland/"
ARUNACHAL = "../../jalend/ParsedData/ArunachalPradesh/"
DELHI = "../DataSets/ElectoralRollsData/ParsedData/Delhi/"
SIKKIM = "../DataSets/ElectoralRollsData/ParsedData/Sikkim/"
GOA = "../DataSets/ElectoralRollsData/ParsedData/GoaParsed/"
MIZORAM = "../DataSets/ElectoralRollsData/ParsedData/Mizoram/"

In [5]:
  from keras.models import load_model
  import re
  import numpy as np
  import pandas as pd

  def vocab():
    alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    special = [' ']
    vocab = alpha+special+['END']

    return vocab
    
  def getIndex(df, maxlen, file):
    voc = vocab()
    char_index = dict((c, i) for i, c in enumerate(voc))
    
    X = []
    next_name = False
    
    # Club all the whitespaces
    
    trunc_X = [str(i).lower()[0:maxlen] for i in df['Name']]
    names = []
    for i in range(len(trunc_X)):
        name = trunc_X[i]
        tmp = []
        for char in str(name):
            if char not in voc:
                with open(file,'a') as f:
                    f.write(name+'\n')
                next_name = True
                break
            else:
                tmp.append(char_index[char])
        if next_name:
            next_name = False
            continue
        for k in range(0,maxlen - len(str(name))):
            tmp.append(char_index["END"])
        X.append(tmp)
        names.append(name)
            
    return np.asarray(X), names

In [6]:
# def returnPred(df, model):
#     df = preprocess_df(df)
#     test_x, names = getIndex(df, 30, 'test_twitter_gender.txt')
#     pred = model.predict(np.asarray(test_x), verbose = 1)
#     y_pred = np.argmax(pred, axis =1)
#     df = pd.DataFrame()
#     df['Name'] = names
#     df['Gender'] = y_pred
#     return df

def returnPredCaste(df, model):
    df = preprocess_df(df)
    test_x, names = getIndex(df, 30, 'test_twitter_gender.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['Caste'] = y_pred
    return df

In [7]:
import re
import numpy as np

def preprocess_df(df):
    df['Name'] = [str(name).lower() for name in df['Name']]
    df['Name'] = [re.sub('[^a-zA-Z]+', ' ', str(name)) for name in df['Name']]
    df['Name'] = [re.sub('[ \t\n]+',' ', str(name)) for name in df['Name']]
    df['Name'].replace('', np.nan, inplace=True)
    df.dropna(subset=['Name'], inplace=True)
    return df

In [10]:
# def get_gender_model():
#   filename = "/content/drive/My Drive/BTP/SavedModels/ERModel"
#   model = load_model(filename)
#   return model

def get_caste_model():
  filename = "/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEData/CNNModel/SavedModel/CasteModel"
  model = load_model(filename)
  return model

In [11]:
# gender_model = get_gender_model()
caste_model = get_caste_model()

2022-01-25 09:42:07.634170: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-25 09:42:07.634274: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: obelix
2022-01-25 09:42:07.634296: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: obelix
2022-01-25 09:42:07.634481: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-01-25 09:42:07.634544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-01-25 09:42:07.634559: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-01-25 09:42:07.635096: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [44]:
def getStateCasteData(files, loc, name):
    req = []
    for file in files:
        if ".csv" in file:
            req.append(file)
    ac_wise_results = []
    for file in req:
        file_data = [file]
        df = pd.read_csv(loc+file, header=0, names=['Name','father_name','husband_name','mother_name','house_number','gender','age'])
        try :
            df = returnPredCaste(df, caste_model)
            l =  list(df.groupby('Caste')['Name'].count())
            res = l[1] 
            gen = l[0]
            file_data.append(res)
            file_data.append(gen)
            l =  df.groupby('gender')['Name'].count()
    #         import pdb;
    #         pdb.set_trace()
            try :
                fem = l['FEMALE']
            except :
                print(l)
                fem = 0
            try :
                mal = l['FEMALE']
            except :
                print(l)
                mal = 0
            file_data.append(fem)
            file_data.append(mal)

            ac_wise_results.append(file_data)
        except :
            with open('/home/souvic/mounted/btp/vahini/er_wrong_files.txt', 'a') as f:
                f.write(file)
#             import pdb;
#             pdb.set_trace()
#         
#         break;

    ac_wise_df = pd.DataFrame(ac_wise_results, columns=['AC', 'Reserved', 'General', 'Female', 'Male'])
    ac_wise_df.to_csv("/home/souvic/mounted/btp/vahini/ERResults/"+name)
    return ac_wise_df

In [31]:
# from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Dropout, Conv2D


In [32]:
# import tensorflow as tf
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session(config=config)

In [33]:
# import os
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


# caste_model.predict(np.asarray(test_x), verbose = 1)
# from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Dropout, Conv2D

In [34]:
# import tensorflow as tf

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [35]:
# import tensorflow as tf

# gpu_devices = tf.config.experimental.list_physical_devices("GPU")
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)
# #     print(device)

In [36]:
# import tensorflow as tf

# print(tf.__version__)

In [37]:
# !cat /usr/include/cudnn_version.h  | grep CUDNN_MAJOR -A 2

In [38]:
# !nvidia-smi

In [65]:
test = pd.read_csv('../ERResults/manipur.csv')
print(test.shape)

(2911, 6)


In [63]:
files = listdir(MANIPUR)

In [64]:
df = getStateCasteData(listdir(MANIPUR), MANIPUR, 'manipur.csv')

15/15 [==============================] - 0s 2ms/step


15/15 [==============================] - 0s 2ms/step


30/30 [==============================] - 0s 2ms/step


11/11 [==============================] - 0s 3ms/step


27/27 [==============================] - 0s 2ms/step


...

17/17 [==============================] - 0s 2ms/step


21/21 [==============================] - 0s 2ms/step


28/28 [==============================] - 0s 2ms/step


29/29 [==============================] - 0s 2ms/step


26/26 [==============================] - 0s 2ms/step


In [60]:
files = listdir(ARUNACHAL)
# print(len(files))

In [61]:
df = getStateCasteData(listdir(ARUNACHAL), ARUNACHAL, 'arunachal.csv')

13/13 [==============================] - 0s 2ms/step
gender
MALE    410
Name: Name, dtype: int64
gender
MALE    410
Name: Name, dtype: int64
7/7 [==============================] - 0s 2ms/step


7/7 [==============================] - 0s 3ms/step


16/16 [==============================] - 0s 2ms/step


8/8 [==============================] - 0s 2ms/step


31/31 [==============================] - 0s 2ms/step
gender
MALE    961
Name: Name, dtype: int64
gender
MALE    961
Name: Name, dtype: int64
5/5 [==============================] - 0s 2ms/step


...

10/10 [==============================] - 0s 2ms/step


17/17 [==============================] - 0s 2ms/step


33/33 [==============================] - 0s 2ms/step


15/15 [==============================] - 0s 2ms/step


9/9 [==============================] - 0s 2ms/step


In [53]:
files = listdir(NAGALAND)

In [55]:
df = getStateCasteData(listdir(NAGALAND), NAGALAND, 'nagaland.csv')

22/22 [==============================] - 0s 2ms/step


30/30 [==============================] - 0s 2ms/step


23/23 [==============================] - 0s 2ms/step


16/16 [==============================] - 0s 2ms/step


28/28 [==============================] - 0s 2ms/step


...

29/29 [==============================] - 0s 2ms/step


19/19 [==============================] - 0s 2ms/step


19/19 [==============================] - 0s 2ms/step


31/31 [==============================] - 0s 2ms/step


8/8 [==============================] - 0s 2ms/step


In [45]:
files = listdir(MEGHALAYA)

In [ ]:
df = getStateCasteData(listdir(MEGHALAYA), MEGHALAYA, 'meghalaya.csv')

20/20 [==============================] - 0s 2ms/step
gender
MALE    629
Name: Name, dtype: int64
gender
MALE    629
Name: Name, dtype: int64
18/18 [==============================] - 0s 2ms/step
gender
MALE    542
Name: Name, dtype: int64
gender
MALE    542
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    453
Name: Name, dtype: int64
gender
MALE    453
Name: Name, dtype: int64
18/18 [==============================] - 0s 2ms/step
gender
MALE    555
Name: Name, dtype: int64
gender
MALE    555
Name: Name, dtype: int64
13/13 [==============================] - 0s 2ms/step
gender
MALE    384
Name: Name, dtype: int64
gender
MALE    384
Name: Name, dtype: int64
40/40 [==============================] - 0s 2ms/step
gender
MALE    1257
Name: Name, dtype: int64
gender
MALE    1257
Name: Name, dtype: int64
24/24 [==============================] - 0s 2ms/step
gender
MALE    753
Name: Name, dtype: int64
gender
MALE    753
Name: Name, dtype: int64
24/24 [====

14/14 [==============================] - 0s 2ms/step
gender
MALE    420
Name: Name, dtype: int64
gender
MALE    420
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    459
Name: Name, dtype: int64
gender
MALE    459
Name: Name, dtype: int64
16/16 [==============================] - 0s 2ms/step
gender
MALE    484
Name: Name, dtype: int64
gender
MALE    484
Name: Name, dtype: int64
19/19 [==============================] - 0s 2ms/step
gender
MALE    576
Name: Name, dtype: int64
gender
MALE    576
Name: Name, dtype: int64
13/13 [==============================] - 0s 2ms/step
gender
MALE    381
Name: Name, dtype: int64
gender
MALE    381
Name: Name, dtype: int64
24/24 [==============================] - 0s 2ms/step
gender
MALE    731
Name: Name, dtype: int64
gender
MALE    731
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    632
Name: Name, dtype: int64
gender
MALE    632
Name: Name, dtype: int64
14/14 [======

27/27 [==============================] - 0s 2ms/step
gender
MALE    833
Name: Name, dtype: int64
gender
MALE    833
Name: Name, dtype: int64
21/21 [==============================] - 0s 2ms/step
gender
MALE    655
Name: Name, dtype: int64
gender
MALE    655
Name: Name, dtype: int64
18/18 [==============================] - 0s 2ms/step
gender
MALE    555
Name: Name, dtype: int64
gender
MALE    555
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    462
Name: Name, dtype: int64
gender
MALE    462
Name: Name, dtype: int64
18/18 [==============================] - 0s 2ms/step
gender
MALE    551
Name: Name, dtype: int64
gender
MALE    551
Name: Name, dtype: int64
21/21 [==============================] - 0s 2ms/step
gender
MALE    637
Name: Name, dtype: int64
gender
MALE    637
Name: Name, dtype: int64
24/24 [==============================] - 0s 3ms/step
gender
MALE    756
Name: Name, dtype: int64
gender
MALE    756
Name: Name, dtype: int64
22/22 [======

31/31 [==============================] - 0s 2ms/step
gender
MALE    962
Name: Name, dtype: int64
gender
MALE    962
Name: Name, dtype: int64
22/22 [==============================] - 0s 2ms/step
gender
MALE    684
Name: Name, dtype: int64
gender
MALE    684
Name: Name, dtype: int64
19/19 [==============================] - 0s 2ms/step
gender
MALE    580
Name: Name, dtype: int64
gender
MALE    580
Name: Name, dtype: int64
19/19 [==============================] - 0s 2ms/step
gender
MALE    591
Name: Name, dtype: int64
gender
MALE    591
Name: Name, dtype: int64
21/21 [==============================] - 0s 2ms/step
gender
MALE    639
Name: Name, dtype: int64
gender
MALE    639
Name: Name, dtype: int64
23/23 [==============================] - 0s 2ms/step
gender
MALE    703
Name: Name, dtype: int64
gender
MALE    703
Name: Name, dtype: int64
14/14 [==============================] - 0s 2ms/step
gender
MALE    423
Name: Name, dtype: int64
gender
MALE    423
Name: Name, dtype: int64
15/15 [======

24/24 [==============================] - 0s 2ms/step
gender
MALE    734
Name: Name, dtype: int64
gender
MALE    734
Name: Name, dtype: int64
13/13 [==============================] - 0s 2ms/step
gender
MALE    399
Name: Name, dtype: int64
gender
MALE    399
Name: Name, dtype: int64
16/16 [==============================] - 0s 2ms/step
gender
MALE    492
Name: Name, dtype: int64
gender
MALE    492
Name: Name, dtype: int64
14/14 [==============================] - 0s 3ms/step
gender
MALE    424
Name: Name, dtype: int64
gender
MALE    424
Name: Name, dtype: int64
16/16 [==============================] - 0s 2ms/step
gender
MALE    504
Name: Name, dtype: int64
gender
MALE    504
Name: Name, dtype: int64
21/21 [==============================] - 0s 2ms/step
gender
MALE    648
Name: Name, dtype: int64
gender
MALE    648
Name: Name, dtype: int64
22/22 [==============================] - 0s 2ms/step
gender
MALE    673
Name: Name, dtype: int64
gender
MALE    673
Name: Name, dtype: int64
16/16 [======

...

14/14 [==============================] - 0s 2ms/step
gender
MALE    431
Name: Name, dtype: int64
gender
MALE    431
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    612
Name: Name, dtype: int64
gender
MALE    612
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    632
Name: Name, dtype: int64
gender
MALE    632
Name: Name, dtype: int64
21/21 [==============================] - 0s 2ms/step
gender
MALE    638
Name: Name, dtype: int64
gender
MALE    638
Name: Name, dtype: int64
28/28 [==============================] - 0s 2ms/step
gender
MALE    864
Name: Name, dtype: int64
gender
MALE    864
Name: Name, dtype: int64
19/19 [==============================] - 0s 2ms/step
gender
MALE    576
Name: Name, dtype: int64
gender
MALE    576
Name: Name, dtype: int64
22/22 [==============================] - 0s 2ms/step
gender
MALE    681
Name: Name, dtype: int64
gender
MALE    681
Name: Name, dtype: int64
23/23 [======

22/22 [==============================] - 0s 2ms/step
gender
MALE    670
Name: Name, dtype: int64
gender
MALE    670
Name: Name, dtype: int64
20/20 [==============================] - 0s 3ms/step
gender
MALE    622
Name: Name, dtype: int64
gender
MALE    622
Name: Name, dtype: int64
24/24 [==============================] - 0s 2ms/step
gender
MALE    734
Name: Name, dtype: int64
gender
MALE    734
Name: Name, dtype: int64
26/26 [==============================] - 0s 2ms/step
gender
MALE    820
Name: Name, dtype: int64
gender
MALE    820
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    618
Name: Name, dtype: int64
gender
MALE    618
Name: Name, dtype: int64
27/27 [==============================] - 0s 2ms/step
gender
MALE    837
Name: Name, dtype: int64
gender
MALE    837
Name: Name, dtype: int64
17/17 [==============================] - 0s 2ms/step
gender
MALE    508
Name: Name, dtype: int64
gender
MALE    508
Name: Name, dtype: int64
16/16 [======

37/37 [==============================] - 0s 2ms/step
gender
MALE    1161
Name: Name, dtype: int64
gender
MALE    1161
Name: Name, dtype: int64
10/10 [==============================] - 0s 2ms/step
gender
MALE    306
Name: Name, dtype: int64
gender
MALE    306
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    460
Name: Name, dtype: int64
gender
MALE    460
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    616
Name: Name, dtype: int64
gender
MALE    616
Name: Name, dtype: int64
17/17 [==============================] - 0s 2ms/step
gender
MALE    525
Name: Name, dtype: int64
gender
MALE    525
Name: Name, dtype: int64
23/23 [==============================] - 0s 2ms/step
gender
MALE    712
Name: Name, dtype: int64
gender
MALE    712
Name: Name, dtype: int64
16/16 [==============================] - 0s 2ms/step
gender
MALE    504
Name: Name, dtype: int64
gender
MALE    504
Name: Name, dtype: int64
18/18 [====

31/31 [==============================] - 0s 2ms/step
gender
MALE    954
Name: Name, dtype: int64
gender
MALE    954
Name: Name, dtype: int64
19/19 [==============================] - 0s 2ms/step
gender
MALE    573
Name: Name, dtype: int64
gender
MALE    573
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    619
Name: Name, dtype: int64
gender
MALE    619
Name: Name, dtype: int64
18/18 [==============================] - 0s 2ms/step
gender
MALE    540
Name: Name, dtype: int64
gender
MALE    540
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    444
Name: Name, dtype: int64
gender
MALE    444
Name: Name, dtype: int64
24/24 [==============================] - 0s 2ms/step
gender
MALE    741
Name: Name, dtype: int64
gender
MALE    741
Name: Name, dtype: int64
23/23 [==============================] - 0s 2ms/step
gender
MALE    697
Name: Name, dtype: int64
gender
MALE    697
Name: Name, dtype: int64
20/20 [======

17/17 [==============================] - 0s 2ms/step
gender
MALE    527
Name: Name, dtype: int64
gender
MALE    527
Name: Name, dtype: int64
13/13 [==============================] - 0s 2ms/step
gender
MALE    388
Name: Name, dtype: int64
gender
MALE    388
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    613
Name: Name, dtype: int64
gender
MALE    613
Name: Name, dtype: int64
20/20 [==============================] - 0s 2ms/step
gender
MALE    620
Name: Name, dtype: int64
gender
MALE    620
Name: Name, dtype: int64
17/17 [==============================] - 0s 2ms/step
gender
MALE    498
Name: Name, dtype: int64
gender
MALE    498
Name: Name, dtype: int64
15/15 [==============================] - 0s 2ms/step
gender
MALE    472
Name: Name, dtype: int64
gender
MALE    472
Name: Name, dtype: int64
22/22 [==============================] - 0s 2ms/step
gender
MALE    697
Name: Name, dtype: int64
gender
MALE    697
Name: Name, dtype: int64
23/23 [======

In [3]:
er_df =  ERData_MajorityLabel()

> /home/souvic/mounted/btp/vahini/Name2Demographics/Models/ERData/PreProcessing/er_preprocess.py(144)ERData_MajorityLabel()
    142     names = dropDuplicates(names)
    143     import pdb; pdb.set_trace()
--> 144     final_df = pd.DataFrame()
    145     final_df['Name'] = names['Name']
    146     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                         Name  Gender  female_freq  male_freq  new_labels  \
7135026                sunita  FEMALE      52740.0      106.0         0.0   
7024311                poonam  FEMALE      37029.0      120.0         0.0   
6943715                 pooja  FEMALE      35158.0      166.0         0.0   
7116009                 anita  FEMALE      34765.0       58.0         0.0   
6901628                 jyoti    MALE      30652.0      196.0         0.0   
...                       ...     ...          ...        ...         ...   
4178983  daya nidh| manchanda    MALE          0.0        1.

In [31]:
daman = ERState_df(DAMAN_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                         Name  Gender  female_freq  male_freq  new_labels
105129               sona bai  FEMALE         23.0        NaN         0.0
106219              lacmi bai  FEMALE         18.0        NaN         0.0
101317               rani bai  FEMALE         17.0        NaN         0.0
101825               mani bai  FEMALE         16.0        1.0         0.0
79043               ratan bai  FEMALE         10.0        NaN         0.0
...                       ...     ...          ...        ...         ...
106898      baria virji jeraj    MALE          NaN        1.0         1.0
106900     baria manish virji    MALE          NaN        1.0         1.0
1069

In [32]:
manipur = ERState_df(MANIPUR_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                   Name  Gender  female_freq  male_freq  new_labels
1151930            mary  FEMALE        606.0        2.0         0.0
1143077          thoibi  FEMALE        449.0        9.0         0.0
1096091           kaini  FEMALE        299.0        1.0         0.0
1096167            losa  FEMALE        299.0        1.0         0.0
1142921         ibemcha  FEMALE        295.0        3.0         0.0
...                 ...     ...          ...        ...         ...
1153806  ramesh chhetri    MALE          NaN        3.0         1.0
1153807   bimol pokhrel    MALE          NaN        1.0         1.0
1153808      darpan rai    MALE          NaN        1.0   

In [33]:
nagaland = ERState_df(NAGALAND_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                    Name  Gender  female_freq  male_freq  new_labels
528589           asangla  FEMALE        748.0        4.0         0.0
525027             monyu  FEMALE        731.0       14.0         0.0
530567              lily  FEMALE        711.0        9.0         0.0
530370              mary  FEMALE        676.0        6.0         0.0
528433            alemla  FEMALE        604.0        8.0         0.0
...                  ...     ...          ...        ...         ...
531300            abaube    MALE          NaN        1.0         1.0
531303  haireudibe neime    MALE          NaN        1.0         1.0
531306          gwangdau    MALE          NaN    

In [34]:
arunachal = ERState_df(ARUNACHAL_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                    Name  Gender  female_freq  male_freq  new_labels
569949     tsering lhamu  FEMALE        168.0        1.0         0.0
570266        lham drema  FEMALE        167.0        NaN         0.0
570919     tsering drema  FEMALE        158.0        2.0         0.0
565732             karmu  FEMALE        146.0        1.0         0.0
570046      tsering pema  FEMALE        145.0        1.0         0.0
...                  ...     ...          ...        ...         ...
575760    moiran moithak    MALE          NaN        1.0         1.0
575761   moiroi! moithak    MALE          NaN        1.0         1.0
575764  ngahang thingnok    MALE          NaN    

In [35]:
sikkim = ERState_df(SIKKIM_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                               Name  Gender  female_freq  male_freq  \
157803                   sunita rai  FEMALE        366.0        1.0   
156488         tshering doma bhutia  FEMALE        308.0        2.0   
159073                 suk maya rai  FEMALE        268.0        2.0   
159491                  doma sherpa  FEMALE        264.0        4.0   
157944                    anita rai  FEMALE        252.0        3.0   
...                             ...     ...          ...        ...   
160718              preetam pradhan    MALE          NaN        6.0   
160720                 simon tamang    MALE          NaN        9.0   
160721              ashrita bas

In [36]:
meghalaya  = ERState_df(MEGHALAYA_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                      Name  Gender  female_freq  male_freq  new_labels
1458228      sabina sangma  FEMALE        127.0        NaN         0.0
1454712       silme sangma  FEMALE        115.0        1.0         0.0
1448784       sabina marak  FEMALE        111.0        2.0         0.0
1415384      momotaz begum  FEMALE        103.0        NaN         0.0
1451921      monowara bibi  FEMALE        102.0        NaN         0.0
...                    ...     ...          ...        ...         ...
1461389   melicha wali ang    MALE          NaN        1.0         1.0
1461393   hibatstar kurbah    MALE          NaN        1.0         1.0
1461394  dilipstar nongbri    M

In [37]:
mizoram = ERState_df(MIZORAM_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                        Name  Gender  female_freq  male_freq  new_labels
286373           lalremruati  FEMALE       2177.0       47.0         0.0
286220           lalmuanpuii  FEMALE       1548.0       36.0         0.0
285726           vanlalruati  FEMALE       1480.0       25.0         0.0
286077            laltanpuii  FEMALE       1438.0       32.0         0.0
286385           lalnunmawii  FEMALE       1417.0       16.0         0.0
...                      ...     ...          ...        ...         ...
286468            lalvarsiem    MALE          NaN        1.0         1.0
286469              khawlkim    MALE          NaN        1.0         1.0
286470       

In [38]:
delhi = ERState_df(DELHI_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                      Name  Gender  female_freq  male_freq  new_labels
2341879             sunita  FEMALE      52491.0      104.0         0.0
2341938             poonam  FEMALE      37006.0      120.0         0.0
2341671              pooja  FEMALE      35146.0      166.0         0.0
2341242              anita  FEMALE      34573.0       58.0         0.0
2339864              jyoti  FEMALE      30640.0      194.0         0.0
...                    ...     ...          ...        ...         ...
2341934   surendraaggarwal    MALE          NaN        1.0         1.0
2341939          dayachand    MALE          NaN       76.0         1.0
2341945     ajay bhatnagar    M

In [58]:
goa = ERState_df(GOA_CSV)

> /tmp/ipykernel_31326/3186903195.py(45)ERState_df()
     43     names = dropDuplicates(names)
     44     import pdb; pdb.set_trace()
---> 45     final_df = pd.DataFrame()
     46     final_df['Name'] = names['Name']
     47     final_df['Gender'] = names['new_labels']

ipdb> names.sort_values(by='female_freq',ascending=False)
                     Name  Gender  female_freq  male_freq  new_labels
726963    maria fernandes  FEMALE        569.0        NaN         0.0
726522         laxmi naik  FEMALE        348.0        1.0         0.0
724121   fatima fernandes  FEMALE        321.0        NaN         0.0
728571        sunita naik  FEMALE        268.0        NaN         0.0
727351        reshma naik  FEMALE        259.0        1.0         0.0
...                   ...     ...          ...        ...         ...
729564       vishnu chari    MALE          NaN       16.0         1.0
729569       vishal gawas    MALE          NaN       24.0         1.0
729571      narayan gawas    MALE       

In [59]:
goa.head()

,Name,Gender
0,naro gaonkar,1.0
1,shwari gaonkar,0.0
2,dhanaji paryekar,1.0
3,dhanashri paryekar,0.0
4,viraj paryekar,1.0


In [41]:
daman1 = pd.read_csv(DAMAN_CSV)


In [49]:
GOA = "../bigdata/ParsedData/GoaParsed/"

In [54]:
files = listdir(GOA)

In [55]:
from os import listdir

#  Goa figure out now
d1 = getStateData(files, GOA)


In [56]:
d1.shape

(1053415, 7)

In [57]:
d1.to_csv(GOA_CSV)

In [ ]:
getStateData(files, loc)

In [2]:
# Correct and update MEghalaya csv

In [7]:
files = listdir(MEGHALAYA)

In [8]:
df = getStateData(listdir(MEGHALAYA), MEGHALAYA)

In [13]:
daman = getStateData(listdir(DAMAN), DAMAN)
daman.to_csv(DAMAN_CSV)

In [14]:
manipur = getStateData(listdir(MANIPUR), MANIPUR)
manipur.to_csv(MANIPUR_CSV)

In [15]:
meghalaya = getStateData(listdir(MEGHALAYA), MEGHALAYA)
meghalaya.to_csv(MEGHALAYA_CSV)

In [17]:
nagaland = getStateData(listdir(NAGALAND), NAGALAND)
nagaland.to_csv(NAGALAND_CSV)

In [19]:
arunachal =getStateData(listdir(ARUNACHAL), ARUNACHAL)
arunachal.to_csv(ARUNACHAL_CSV)

In [16]:
delhi = getStateData(listdir(DELHI), DELHI)
delhi.to_csv(DELHI_CSV)

In [20]:
sikkim = getStateData(listdir(SIKKIM), SIKKIM)
sikkim.to_csv(SIKKIM_CSV)

In [21]:
mizoram = getStateData(listdir(MIZORAM), MIZORAM)
mizoram.to_csv(MIZORAM_CSV)

In [22]:
meg = pd.read_csv(MEGHALAYA_CSV)

In [23]:
meg.head()

,Unnamed: 0,name,father_name,husband_name,mother_name,house_number,gender,age
0,0,AILAD SUCHIANG,SDIH TYNGKAN,NaN,NaN,1,FEMALE,70
1,1,RIIEl HOK SUCHIANG,NaN,NaN,AILAD SUCHIANG,1,FEMALE,29
2,2,RIDALIN SUCHIANG,SHIA BIAM,NaN,NaN,1,FEMALE,42
3,3,MARDEIMON SUCHIANG,ERAS WORLOI,NaN,NaN,1,FEMALE,25
4,4,DOLBI SUCHIANG,NaN,BNAl SHADAP,NaN,2,FEMALE,52


In [24]:
meg['name'] = [ name.replace('Other\'s : ', ' ') for name in meg['name']]

In [32]:
meg.to_csv(MEGHALAYA_CSV)

In [33]:
meg.head()

,name,father_name,husband_name,mother_name,house_number,gender,age
0,AILAD SUCHIANG,SDIH TYNGKAN,NaN,NaN,1,FEMALE,70
1,RIIEl HOK SUCHIANG,NaN,NaN,AILAD SUCHIANG,1,FEMALE,29
2,RIDALIN SUCHIANG,SHIA BIAM,NaN,NaN,1,FEMALE,42
3,MARDEIMON SUCHIANG,ERAS WORLOI,NaN,NaN,1,FEMALE,25
4,DOLBI SUCHIANG,NaN,BNAl SHADAP,NaN,2,FEMALE,52
